In [1]:
import numpy as np
import pandas as pd

In [9]:
subject = pd.read_csv('subject-info.csv')
test = pd.read_csv('test_measure.csv')

In [13]:
print(test.head())
print(subject.head())
print(subject.tail())
print(test.tail())

   time  Speed    HR    VO2   VCO2    RR    VE ID_test  ID
0     0    5.0  63.0  478.0  360.0  27.0  13.3     2_1   2
1     2    5.0  75.0  401.0  295.0  23.0  10.3     2_1   2
2     4    5.0  82.0  449.0  319.0  29.0  12.2     2_1   2
3     7    5.0  87.0  461.0  340.0  28.0  12.8     2_1   2
4     9    5.0  92.0  574.0  417.0  28.0  14.6     2_1   2
    Age  Weight  Height  Humidity  Temperature  Sex   ID ID_test
0  10.8    48.8   163.0      39.0         20.7    1  543   543_1
1  11.8    41.0   150.0      41.0         22.3    1   11    11_1
2  12.2    46.0   160.0      37.0         21.5    0  829   829_1
3  13.2    71.0   190.0      49.0         23.8    1  284   284_1
4  13.7    53.8   169.7      40.0         25.3    0  341   341_1
      Age  Weight  Height  Humidity  Temperature  Sex   ID ID_test
987  59.1    64.7   172.0      38.0         24.4    0  856   856_2
988  59.7    65.2   172.0      51.0         16.8    0  856   856_3
989  61.3   102.0   185.0      56.0         20.5    0  